In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Set up Selenium options and driver
options = Options()
options.add_argument("--disable-gpu")
# options.add_argument("--headless")  # Optional: Run in headless mode

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Navigate to the page
base_url = "https://www.nykaa.com/brands/nykaa-naturals/c/7666?"
params = "ptype=brand&id=7666&root=brand_menu,top_brands,Nykaa%20Naturals"
paginationNumber = 1

# Allow initial time for the page to load
time.sleep(5)

# Scroll incrementally until all products are loaded
scroll_pause_time = 2  # Wait time after each scroll

# Lists to store data
productLinks = []
products = []

while True:

    if paginationNumber == 1:
        url = f"{base_url}{params}"
    else:
        url = f"{base_url}page_no={paginationNumber}&sort=popularity&ptype=brand&id=7666&root=brand_menu,top_brands,Nykaa%20Naturals&eq=desktop"
    
    # Load the Page
    driver.get(url)
    print(f"Scraping Page {paginationNumber}: {url}")
    time.sleep(3)
    
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

    # Wait until the products are loaded
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.productWrapper.css-17nge1h'))
        )
    except TimeoutException:
        print(f"Scraped the Total number of pages: {paginationNumber - 1}")
        break

    # Parse the page content with BeautifulSoup
    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')
    
    # Find all product items
    deygaOrganicProducts = soup.findAll('div', class_='productWrapper css-17nge1h')
    if not deygaOrganicProducts:
        print("No Products found on this page.")
        break

    # Extract product links
    for product in deygaOrganicProducts:
        # Extract the Stock Status
        stockTag = product.find('span', class_ = "css-lg5xc9")
        if stockTag:
            stock = stockTag.get_text(strip = True)
        else:
            stock = "In Stock"

        # Extract the Product Highlight
        highlightTag = product.find("li", class_ = "custom-tag css-1bse542")
        if highlightTag:
            highlight = highlightTag.get_text(strip = True)
        else:
            highlight = "None"

        # Extract product link
        linkTag = product.find('a', href=True)
        if linkTag:
            productLink = linkTag['href']
            # Check if the link is a relative URL and add the base URL if necessary
            if not productLink.startswith("http"):  # Relative URL
                productLink = "https://www.nykaa.com" + productLink
            productLinks.append({"link" : productLink, "stock" : stock, "highlight" : highlight})
                
    for productData in productLinks:
        
        link = productData["link"]
        stock = productData["stock"]
        highlight = productData["highlight"]
        
        driver.get(link)
        time.sleep(3)  # Allow time for the page to load

        # Parse individual product page
        productPageContent = driver.page_source
        productSoup = BeautifulSoup(productPageContent, 'html.parser')
    
        # Extract Product Name
        productTag = productSoup.find('h1', class_='css-1gc4x7i')
        if productTag:
            prodName = productTag.get_text(strip=True)
        else:
            prodName = "No Description Available"
    
        # Extract Ratings
        ratingTag = productSoup.find('div', class_="css-1m0y15j")
        if ratingTag:
            ratingText = ratingTag.find('div', class_="css-m6n3ou")
            if ratingText:
                ratings = ratingText.get_text(strip = True)
            else:
                ratings = "N/A"
        else:
            ratings = "N/A"

        # Extract the Ratings & Reviews Count
        ratingCountTag = productSoup.find('div', class_ = "css-1eip5u4")
        if ratingCountTag:
            ratingCount = ratingCountTag.get_text(strip = True)
        else:
            ratingCount = "None"

        # Extract the Offer Price and Original Price
        productDiv = productSoup.find('div', class_ = "css-1d0jf8e")

        # Extract the first two (span) tags
        if productDiv:
            spanTags = productDiv.find_all('span')
            # print(len(spanTags))

            originalPrice = "N/A"
            offerPrice = "N/A"

            if len(spanTags) == 2:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = "N/A"
            elif len(spanTags) == 4:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = spanTags[2].get_text(strip = True)

            # Print the Values
            # print(f"Original Price: {originalPrice}, Offer Price: {offerPrice}")

        else:
            print("Product div not found, defaulting to Unavailable for prices")

        # Extract the Discount
        discountTag = productSoup.find('span', class_ = "css-bhhehx")
        if discountTag:
            discount = discountTag.get_text(strip = True)
        else:
            discount = "No Discount"

        products.append({
            "Product Brand" : "Nykaa Naturals",
            "Product Name" : prodName,
            "Product Rating" : ratings,
            "Product Rating & Review Count" : ratingCount,
            "Product Original Price" : originalPrice, 
            "Product Offer Price" : offerPrice, 
            "Product Discount" : discount,
            "Product Highlight" : highlight,
            "Product Stock Status" : stock
        })

    # Clear the product links list to avoid the duplication
    productLinks.clear()

    paginationNumber += 1 # Increment the Page Number
    time.sleep(5)

# Close the browser after scraping
driver.quit()

Scraping Page 1: https://www.nykaa.com/brands/nykaa-naturals/c/7666?ptype=brand&id=7666&root=brand_menu,top_brands,Nykaa%20Naturals
Scraping Page 2: https://www.nykaa.com/brands/nykaa-naturals/c/7666?page_no=2&sort=popularity&ptype=brand&id=7666&root=brand_menu,top_brands,Nykaa%20Naturals&eq=desktop
Scraping Page 3: https://www.nykaa.com/brands/nykaa-naturals/c/7666?page_no=3&sort=popularity&ptype=brand&id=7666&root=brand_menu,top_brands,Nykaa%20Naturals&eq=desktop
Scraping Page 4: https://www.nykaa.com/brands/nykaa-naturals/c/7666?page_no=4&sort=popularity&ptype=brand&id=7666&root=brand_menu,top_brands,Nykaa%20Naturals&eq=desktop
Scraping Page 5: https://www.nykaa.com/brands/nykaa-naturals/c/7666?page_no=5&sort=popularity&ptype=brand&id=7666&root=brand_menu,top_brands,Nykaa%20Naturals&eq=desktop
Scraped the Total number of pages: 4


In [3]:
print(f"Products: \n {products}")

Products: 
 [{'Product Brand': 'Nykaa Naturals', 'Product Name': 'Nykaa Naturals Onion & Fenugreek Shampoo, Conditioner & Hair Mask - Hair Growth Combo', 'Product Rating': '4.2/5', 'Product Rating & Review Count': '17459ratings&1556reviews', 'Product Original Price': '₹1397', 'Product Offer Price': '₹699', 'Product Discount': '50% Off', 'Product Highlight': 'None', 'Product Stock Status': 'In Stock'}, {'Product Brand': 'Nykaa Naturals', 'Product Name': 'Nykaa Naturals Skin Secrets Bestseller Sheet Mask - Monthly Pack(5 pcs)', 'Product Rating': '4.5/5', 'Product Rating & Review Count': '355641ratings&24180reviews', 'Product Original Price': '₹500', 'Product Offer Price': '₹325', 'Product Discount': '35% Off', 'Product Highlight': 'BESTSELLER', 'Product Stock Status': 'In Stock'}, {'Product Brand': 'Nykaa Naturals', 'Product Name': 'Nykaa Naturals Saffron & Honey Face Scrub for Glowing Skin(100gm)', 'Product Rating': '4.3/5', 'Product Rating & Review Count': '15936ratings&860reviews', 'P

In [4]:
nykaaNaturals = pd.DataFrame(products)
nykaaNaturals

,Product Brand,Product Name,Product Rating,Product Rating & Review Count,Product Original Price,Product Offer Price,Product Discount,Product Highlight,Product Stock Status
0,Nykaa Naturals,"Nykaa Naturals Onion & Fenugreek Shampoo, Cond...",4.2/5,17459ratings&1556reviews,₹1397,₹699,50% Off,None,In Stock
1,Nykaa Naturals,Nykaa Naturals Skin Secrets Bestseller Sheet M...,4.5/5,355641ratings&24180reviews,₹500,₹325,35% Off,BESTSELLER,In Stock
2,Nykaa Naturals,Nykaa Naturals Saffron & Honey Face Scrub for ...,4.3/5,15936ratings&860reviews,₹249,₹137,45% Off,None,In Stock
3,Nykaa Naturals,Nykaa Naturals Skin Secrets Potent Ingredients...,4.5/5,355585ratings&24176reviews,₹500,₹325,35% Off,None,In Stock
4,Nykaa Naturals,Nykaa Naturals Fermented Rice Water & Bamboo S...,4.2/5,11146ratings&753reviews,₹798,₹399,50% Off,None,In Stock
...,...,...,...,...,...,...,...,...,...
66,Nykaa Naturals,Nykaa Soap Story Tea Tree & Aloe Handcrafted S...,4.2/5,1556ratings&410reviews,₹350,N/A,No Discount,None,Out Of Stock
67,Nykaa Naturals,Nykaa Naturals Rosemary Oil & Niacinamide Sulp...,N/A,None,₹349,N/A,No Discount,None,Out Of Stock
68,Nykaa Naturals,Nykaa 3 Ply Mask With Face Shield(1 pcs),2.9/5,112ratings&15reviews,₹75,N/A,No Discount,None,Out Of Stock
69,Nykaa Naturals,Nykaa Naturals Lavender & Chamomile Face Scrub...,4.3/5,15936ratings&860reviews,₹249,₹137,45% Off,None,Out Of Stock


In [5]:
# nykaaNaturals.to_csv("C:/Users/Logiya Vidhyapathy/Documents/KGISL Data Science/Project Files and Documents/Capstone Project/Nykaa Naturals Products_Nykaa.csv", index = False)